<a href="https://colab.research.google.com/github/SARA3SAEED/LLM-2/blob/main/14Day_3_Foundation_FAISS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Read Data

In [ ]:

import requests
import os

data_url = "https://raw.githubusercontent.com/jamescalam/data/main/sentence_embeddings_15K/"
s=[]
# create data directory to store data
if not os.path.exists('./data'):
    os.mkdir('./data')

# download the numpy binary files (dense vectors)
for i in range(57):
    if i < 10:
        i = '0' + str(i)
    res = requests.get(data_url+f"embeddings_{i}.npy")

    with open(f'./data/embeddings_{i}.npy', 'wb') as fp:
        for chunk in res:
            fp.write(chunk)

    print('.', end='')

# and download the respective text file
res = requests.get(f"{data_url}sentences.txt")
with open(f"./data/sentences.txt", 'wb') as fp:
    for chunk in res:
        fp.write(chunk)

.........................................................

In [ ]:
urls = [
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.train.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2013/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/images.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2015/images.test.tsv'
]

In [ ]:
import pandas as pd

import requests
from io import StringIO
res = requests.get('https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/sick2014/SICK_train.txt')
# create dataframe
data = pd.read_csv(StringIO(res.text), sep='\t')
data.head()

,pair_ID,sentence_A,sentence_B,relatedness_score,entailment_judgment
0,1,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,4.5,NEUTRAL
1,2,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,3.2,NEUTRAL
2,3,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,4.7,ENTAILMENT
3,5,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,3.4,NEUTRAL
4,9,The young boys are playing outdoors and the ma...,A group of kids is playing in a yard and an ol...,3.7,NEUTRAL


In [ ]:
# we take all samples from both sentence A and B
sentences = data['sentence_A'].tolist()
sentences[:5]

['A group of kids is playing in a yard and an old man is standing in the background',
 'A group of children is playing in the house and there is no man standing in the background',
 'The young boys are playing outdoors and the man is smiling nearby',
 'The kids are playing outdoors near a man with a smile',
 'The young boys are playing outdoors and the man is smiling nearby']

In [ ]:
# we take all samples from both sentence A and B
sentences = data['sentence_A'].tolist()
sentence_b = data['sentence_B'].tolist()
sentences.extend(sentence_b)  # merge them
len(set(sentences))  # together we have ~4.5K unique sentences

4802

In [ ]:
sentences = data['sentence_A'].tolist()
sentence_b = data['sentence_B'].tolist()
sentences.extend(sentence_b)
len(set(sentences))

4802

In [ ]:
sentences[:5]

['A group of kids is playing in a yard and an old man is standing in the background',
 'A group of children is playing in the house and there is no man standing in the background',
 'The young boys are playing outdoors and the man is smiling nearby',
 'The kids are playing outdoors near a man with a smile',
 'The young boys are playing outdoors and the man is smiling nearby']

In [ ]:


# each of these dataset have the same structure, so we loop through each creating our sentences data
for url in urls:
    res = requests.get(url)
    # extract to dataframe

    smalldata = pd.read_csv(StringIO(res.text), sep='\t', header=None, on_bad_lines='skip')
    # data=data.append(smalldata,ignore_index=False)
    # add to columns 1 and 2 to sentences list
    sentences.extend(smalldata[1].tolist())
    sentences.extend(smalldata[2].tolist())

In [ ]:
for url in urls:
    res = requests.get(url)

    smalldata = pd.read_csv(StringIO(res.text), sep='\t', header=None, on_bad_lines='skip')

    sentences.extend(smalldata[1].tolist())
    sentences.extend(smalldata[2].tolist())

In [ ]:
# smalldata.head()

In [ ]:
# remove duplicates and NaN
sentences = [word for word in list(set(sentences)) if type(word) is str]

In [ ]:
sentences = [word for word in list(set(sentences)) if type(word) is str]

In [ ]:
len(sentences)

14504

In [ ]:
sentences[:5]

['hurt, feel physical pain',
 "Excluding patent litigation, RIM's loss for the quarter was $700,000, or 1 cent per share.",
 'cover or swamp with water.',
 'A black dog running through the snow.',
 'A motorbike rider is running over a man ']

In [ ]:
import numpy as np
!pip install faiss-gpu #cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 6.8 MB/s eta 0:00:00


# raw text > into Embeddings [using an encoder] > save it in FAISS > raw text query > into an Embedding > Euclidean distance to retrieve the text closest to the query

In [ ]:
!pip install -U sentence-transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
# !git clone https://github.com/jamescalam/data.git data-embeddings

Cloning into 'data-embeddings'...
remote: Enumerating objects: 134, done.
remote: Total 134 (delta 0), reused 0 (delta 0), pack-reused 134
Receiving objects: 100% (134/134), 79.25 MiB | 31.51 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [ ]:
# path = '/content/data-embeddings/sentence_embeddings_15K/'
# sentence_embeddings = []
# for i in range(0,57):
#     # if i < 10:
#     #     i = '0' + str(i)
#     res = path+f"embeddings_{i}.npy"
#     print(res)
#     sm = np.load(res)
#     sentence_embeddings.append(sm)

In [ ]:
# arr = np.concatenate(sentence_embeddings)

In [ ]:
# arr.shape

In [ ]:
# sentences = sentences[:100]

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

In [ ]:
from sentence_transformers import SentenceTransformer
# initialize sentence transformer model
#model = SentenceTransformer('bert-base-nli-mean-tokens') #encoding model
model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
len(sentences)

14504

In [ ]:
%%time
sentence_embeddings = model.encode(sentences)
sentence_embeddings.shape

CPU times: user 16min 43s, sys: 7.05 s, total: 16min 50s
Wall time: 18min 13s


(14504, 768)

In [ ]:
%%time
sentence_embeddings = model.encode(sentences)
sentence_embeddings.shape

In [ ]:
sentences[0]

'People are walking outside the building that has several murals on it'

In [ ]:
sentence_embeddings[0]

array([ 1.82786584e-02, -1.62554458e-02,  2.46978831e-02,  1.69377576e-03,
        4.74095121e-02,  2.64739264e-02, -3.67230475e-02,  4.80472594e-02,
        7.31662065e-02, -5.41171292e-03,  2.15139743e-02,  4.57525775e-02,
        1.69058517e-02, -3.51805873e-02,  5.39250597e-02, -3.38780284e-02,
        7.21550873e-03,  3.21253128e-02,  3.28799970e-02, -6.78706262e-03,
       -1.82974916e-02,  4.12828587e-02, -4.40429039e-02,  2.92378142e-02,
       -3.34008820e-02, -4.70056906e-02,  4.21739817e-02,  1.06620230e-02,
       -3.52034857e-03, -4.35390063e-02, -4.73726355e-02, -5.05080307e-03,
       -8.12924409e-04, -7.33982176e-02,  1.70087174e-06,  3.36495519e-04,
       -3.06700524e-02, -3.66430096e-02,  6.39753463e-03,  9.55050625e-03,
        1.32885240e-02, -1.36095032e-01,  5.92184216e-02,  2.37759668e-03,
        5.88795170e-03, -6.97008893e-02,  4.52368557e-02,  3.25695798e-02,
       -6.90768007e-03,  2.73740720e-02, -2.21126545e-02,  3.66752478e-03,
       -4.81269434e-02, -

In [ ]:
# with open(f'./sim_sentences/embeddings_X.npy', 'wb') as fp:
#     np.save(fp, sentence_embeddings[0:256])

In [ ]:
# # saving data
# split = 256
# file_count = 0
# for i in range(0, sentence_embeddings.shape[0], split):
#     end = i + split
#     if end > sentence_embeddings.shape[0] + 1:
#         end = sentence_embeddings.shape[0] + 1
#     file_count = '0' + str(file_count) if file_count < 0 else str(file_count)
#     with open(f'./sim_sentences/embeddings_{file_count}.npy', 'wb') as fp:
#         np.save(fp, sentence_embeddings[i:end, :])
#     print(f"embeddings_{file_count}.npy | {i} -> {end}")
#     file_count = int(file_count) + 1

In [ ]:
import faiss
# sentence_embeddings = arr

In [ ]:
d = sentence_embeddings.shape[1]
d

768

In [ ]:
d = sentence_embeddings.shape[1]
d

In [ ]:
sentences[0]

'People are walking outside the building that has several murals on it'

In [ ]:
sentences[0]

'hurt, feel physical pain'

In [ ]:
sentence_embeddings[0].shape

(768,)

In [ ]:
sentence_embeddings[0].shape

(768,)

In [ ]:
faiss.Index

faiss.swigfaiss.Index

In [ ]:
index = faiss.IndexFlatL2(d)

In [ ]:
index = faiss.IndexFlatL2(d) #euclidean method.  - intialize

In [ ]:
index.is_trained

In [ ]:
index.is_trained

True

In [ ]:
index.add(sentence_embeddings)

In [ ]:
index.add(sentence_embeddings)

In [ ]:
index.ntotal

14504

converted raw text into embeddings > saved the embeddings into FAISS and we are going to use Euclidean distance to measure the distances

In [ ]:
sentences[100:101]

['an assertion offered as evidence, a line of reasoning']

In [ ]:
sentences[100:101]

In [ ]:
k = 10
xq = model.encode(["Someone is performing a dance admidst the rainfall."])

In [ ]:
k = 10
xq = model.encode(["Someone is performing a dance admidst the rainbow"])

Converted the query into an embedding and now we will run Euclidean search on the entire corpus and compare it to the query

In [ ]:
# index

In [ ]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[10388  6289  9611   775  8462 11462  2182  8112  8069 11779]]
CPU times: user 15.3 ms, sys: 931 µs, total: 16.3 ms
Wall time: 19.9 ms


In [ ]:
%%time
D, I = index.search(xq, k)
print(I)

[[ 8336  2797  2308  9694 14433  1611 13119 12675  1754  4209]]
CPU times: user 23.4 ms, sys: 992 µs, total: 24.4 ms
Wall time: 25.1 ms


In [ ]:
[f'{i}: {sentences[i]}' for i in I[0]]

['10388: A person is dancing in the rain',
 '6289: A person is dancing',
 '9611: A man is dancing in the rain',
 '775: A woman is performing in the rain',
 '8462: A male is dancing',
 '11462: A man is dancing',
 '2182: The man is dancing',
 '8112: A girl dances on a sidewalk.',
 '8069: A hiker is on top of the mountain and is dancing',
 '11779: A woman is dancing']

In [ ]:
[f'{i}:{ sentences[i]}' for i in I[0]]

['8336:A person is dancing',
 '2797:A man dances on a colorful wall.',
 '2308:A man is dancing',
 '9694:A woman is staging a dance',
 '14433:The man is dancing',
 '1611:A male is dancing',
 '13119:A hiker is on top of the mountain and is doing a joyful dance',
 '12675:The man is wildly dancing',
 '1754:A dancer posing for the camera in a red and white dress.',
 '4209:A woman is dancing gracefully']

# Quantization

In [ ]:
nlist = 100
quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFFlat(quantizer, d, nlist)

In [ ]:
nlist = 100
quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFFlat(quantizer, d, nlist)

In [ ]:
index.is_trained

False

In [ ]:
index.is_trained

False

In [ ]:
index.train(sentence_embeddings)
index.is_trained

True

In [ ]:
index.train(sentence_embeddings)
index.is_trained

True

In [ ]:
index.add(sentence_embeddings)
index.ntotal

14504

In [ ]:
index.add(sentence_embeddings)
index.ntotal

14504

In [ ]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[ 6289  8462 11462  2182 14308  2986 13386  5437  2805  9658]]
CPU times: user 661 µs, sys: 0 ns, total: 661 µs
Wall time: 648 µs


In [ ]:
%%time
D, I = index.search(xq, k)
print(I)

[[ 8336  7352 12614 13460  1408  3963  3234  3831  3653  8914]]
CPU times: user 1.51 ms, sys: 6 µs, total: 1.52 ms
Wall time: 1.03 ms


In [ ]:
index.nprobe = 10

In [ ]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[10388  6289  9611  8462 11462  2182  8112 11779 13628  2342]]
CPU times: user 3.27 ms, sys: 0 ns, total: 3.27 ms
Wall time: 2.79 ms


['582: A person is dancing in the rain',
 '7999: A man is dancing in the rain',
 '14299: A woman is performing in the rain',
 '8088: The dancer is dancing in front of the sound equipment']

In [ ]:
[f'{i}: {sentences[i]}' for i in I[0]]

['10388: A person is dancing in the rain',
 '6289: A person is dancing',
 '9611: A man is dancing in the rain',
 '8462: A male is dancing',
 '11462: A man is dancing',
 '2182: The man is dancing',
 '8112: A girl dances on a sidewalk.',
 '11779: A woman is dancing',
 '13628: A woman is staging a dance',
 '2342: A girl wearing a pink coat splashes in rain puddles.']

#Homework: implement all the versions mentioned in this article: Comprehensive Guide To Approximate Nearest Neighbors Algorithms | by Eyal Trabelsi | Towards Data Science

In [ ]:
index.make_direct_map()

Technique > Time > Subjective performance